In [1]:
import sys; sys.path.append("../")

In [2]:
from dataset_wrappper import NewsData
from utils_train import transfer_batch_to_device
import os
from run_validation import load_model_for_eval
from pathlib import Path
import matplotlib.pyplot as plt
from transformers import RobertaTokenizerFast
import torch
import torch.nn.functional as F
from utils_evaluation import tokenizer_batch_decode
import numpy as np
import pickle
import copy
import tqdm
import pandas as pd

plt.style.reload_library()
plt.style.use('thesis_style')

%matplotlib inline

In [3]:
BATCH_SIZE = 64
DEVICE = "cuda:0"
CHECKPOINT_TYPE = "best" # else "best"
RESULT_DIR = Path("result-files")

In [4]:
# data = NewsData(batch_size=BATCH_SIZE, tokenizer_name="roberta", dataset_name="ptb_text_only", max_seq_len=64)
# validation_loader = data.val_dataloader(batch_size=BATCH_SIZE)
# tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# print(f"{len(validation_loader)} batches with batch size {BATCH_SIZE}")

In [5]:
def get_clean_name(run_name):
    latent_size = run_name.split("-")[4][-2:]
    if "autoencoder" in run_name:
        FB = "autoencoder"
    else:
        FB = run_name.split("-")[6]
        if len(FB) == 3:
            FB += "0"
        FB = "FB-" + FB
    clean_name = f"NZ-{latent_size} | {FB}"
    return clean_name

In [6]:
PTB_run_name_paths = {}
for r in os.listdir("/home/cbarkhof/code-thesis/NewsVAE/Runs"):
    if "PTB" in r:
        path = Path("/home/cbarkhof/code-thesis/NewsVAE/Runs") / r / f"checkpoint-{CHECKPOINT_TYPE}.pth"
        PTB_run_name_paths[r] = path
        print(path)

/home/cbarkhof/code-thesis/NewsVAE/Runs/2021-02-03-PTB-latent32-FB-0.5-run-09:31:02/checkpoint-best.pth
/home/cbarkhof/code-thesis/NewsVAE/Runs/2021-02-03-PTB-latent32-FB-1.0-run-11:43:17/checkpoint-best.pth
/home/cbarkhof/code-thesis/NewsVAE/Runs/2021-02-03-PTB-latent32-FB-1.50-run-12:13:36/checkpoint-best.pth
/home/cbarkhof/code-thesis/NewsVAE/Runs/2021-02-03-PTB-latent64-FB-1.0-run-13:06:00/checkpoint-best.pth
/home/cbarkhof/code-thesis/NewsVAE/Runs/2021-02-03-PTB-latent32-FB-0.00-run-14:32:09/checkpoint-best.pth
/home/cbarkhof/code-thesis/NewsVAE/Runs/2021-02-03-PTB-latent32-autoencoder-run-17:30:41/checkpoint-best.pth
/home/cbarkhof/code-thesis/NewsVAE/Runs/2021-02-03-PTB-latent64-FB-0.50-run-12:29:58/checkpoint-best.pth
/home/cbarkhof/code-thesis/NewsVAE/Runs/2021-02-03-PTB-latent64-FB-1.50-run-13:22:14/checkpoint-best.pth
/home/cbarkhof/code-thesis/NewsVAE/Runs/2021-02-03-PTB-latent64-FB-0.00-run-17:14:10/checkpoint-best.pth
/home/cbarkhof/code-thesis/NewsVAE/Runs/2021-02-03-PTB

In [13]:
MODE = "AR"

# Exact Match

In [14]:
exact_match_results = pickle.load( open( RESULT_DIR / f"result_exact_match_{MODE}.pickle", "rb" ) )

acc_results = {}
for n, r in exact_match_results.items():
    acc_results[n] = {
        "acc_mean": np.mean(r["exact_match_acc"]),
        "acc_std": np.std(r["exact_match_acc"])
    }
    

df_acc = pd.DataFrame(acc_results).transpose()
df_acc["Accuracy"] = df_acc["acc_mean"].map(lambda x: "{:.2f}".format(x)) + df_acc["acc_std"].map(lambda x: " \tiny{{({:.2f})}}".format(x))
df_acc.index = df_acc.index.map(lambda x: get_clean_name(x))
df_acc = df_acc.reset_index()
df_acc = df_acc[["index", "Accuracy"]]

# PPL

In [15]:
# PERPLEXITY

df_ppl = pd.read_csv(RESULT_DIR / f"perplexity_results_{MODE}.csv", index_col=0)
df_ppl.index.name = None
df_ppl = df_ppl[["index", "Perplexity"]]
df_ppl

,index,Perplexity
0,NZ-64 | FB-0.25,6.18e+03 \tiny{(9.63e+02)}
1,NZ-32 | FB-0.25,6.69e+03 \tiny{(9.33e+02)}
2,NZ-32 | FB-0.50,8.03e+03 \tiny{(1.40e+03)}
3,NZ-64 | FB-0.50,9.76e+03 \tiny{(1.83e+03)}
4,NZ-32 | FB-0.75,1.08e+04 \tiny{(1.98e+03)}
5,NZ-32 | FB-1.00,1.15e+04 \tiny{(2.45e+03)}
6,NZ-64 | FB-0.75,1.33e+04 \tiny{(3.17e+03)}
7,NZ-32 | FB-1.50,1.47e+04 \tiny{(3.38e+03)}
8,NZ-64 | FB-1.00,1.49e+04 \tiny{(3.50e+03)}
9,NZ-64 | FB-1.50,2.64e+04 \tiny{(6.64e+03)}


# AU

In [16]:
# ACTIVE UNITS

AU_results = pickle.load( open( RESULT_DIR / f"result_active_units.pickle", "rb" ) )
df_au = pd.DataFrame(AU_results).transpose()[["active_units"]]
df_au.index = df_au.index.map(lambda x: get_clean_name(x))
df_au = df_au.reset_index()

df_au["Latent size"] = df_au["index"].map(lambda x: x[3:5])
df_au["Active Units"] = df_au["active_units"].map(lambda x: "{}".format(x)) + " / " + df_au["Latent size"]
df_au = df_au[["index", "Active Units"]]
df_au

,index,Active Units
0,NZ-32 | FB-0.50,32 / 32
1,NZ-32 | FB-1.00,32 / 32
2,NZ-32 | FB-1.50,32 / 32
3,NZ-64 | FB-1.00,64 / 64
4,NZ-32 | FB-0.00,0 / 32
5,NZ-32 | autoencoder,32 / 32
6,NZ-64 | FB-0.50,64 / 64
7,NZ-64 | FB-1.50,64 / 64
8,NZ-64 | FB-0.00,0 / 64
9,NZ-64 | autoencoder,64 / 64


In [17]:
# RATE, DISTORTION, ELBO

rate_dist_results = pickle.load( open( RESULT_DIR / f"result_rate_distortion_{MODE}.pickle", "rb" ) )
sum_rate_dist_results = {}
for name, r in rate_dist_results.items():
    elbo, rate, dist = r["elbo"], r["rate"], r["distortion"]
    sum_rate_dist_results[get_clean_name(name)] = {"rate_mean": np.mean(rate), "rate_std": np.std(rate),
                                                   "dist_mean": np.mean(dist), "dist_std": np.std(dist),
                                                   "-elbo_mean": np.mean(elbo), "-elbo_std": np.std(elbo)}

def add_string_cols(df):
    df["Distortion"] = df["dist_mean"].map('{:,.2f}'.format) + df["dist_std"].map(' \tiny{{({:,.2f})}}'.format)
    df["Rate"] = df["rate_mean"].map('{:,.2f}'.format) + df["rate_std"].map(' \tiny{{({:,.2f})}}'.format)
    df["-ELBO"] = df["-elbo_mean"].map('{:,.2f}'.format) + df["-elbo_std"].map(' \tiny{{({:,.2f})}}'.format)
    df = df.sort_values('-elbo_mean')
    return df

df_rate = pd.DataFrame(sum_rate_dist_results).transpose().reset_index()
df_rate = add_string_cols(df_rate)
df_rate = df_rate[["index", "Distortion", "Rate", "-ELBO"]]

In [18]:
import numpy as np

def get_target_rate(x):
    if "autoencoder" in x:
        return np.inf
    else:
        latent_size = int(x[3:5])
        FB = float(x[-4:])
        return float(latent_size) * FB
    
df_rate["Target rate (FB x NZ)"] = df_rate["index"].map(lambda x: get_target_rate(x))
df_rate

,index,Distortion,Rate,-ELBO,Target rate (FB x NZ)
11,NZ-32 | FB-0.25,286.38 \tiny{(26.06)},6.01 \tiny{(0.08)},292.38 \tiny{(26.09)},8.0
13,NZ-64 | FB-0.25,284.03 \tiny{(27.61)},11.09 \tiny{(0.14)},295.12 \tiny{(27.71)},16.0
0,NZ-32 | FB-0.50,295.39 \tiny{(30.12)},13.10 \tiny{(0.17)},308.49 \tiny{(30.22)},16.0
4,NZ-32 | FB-0.00,308.59 \tiny{(29.11)},0.00 \tiny{(0.00)},308.59 \tiny{(29.11)},0.0
8,NZ-64 | FB-0.00,309.21 \tiny{(29.17)},0.00 \tiny{(0.00)},309.22 \tiny{(29.17)},0.0
10,NZ-32 | FB-0.75,298.88 \tiny{(31.44)},20.52 \tiny{(0.26)},319.40 \tiny{(31.57)},24.0
6,NZ-64 | FB-0.50,296.16 \tiny{(29.72)},24.45 \tiny{(0.34)},320.61 \tiny{(29.92)},32.0
1,NZ-32 | FB-1.00,297.86 \tiny{(33.61)},29.51 \tiny{(0.36)},327.37 \tiny{(33.80)},32.0
2,NZ-32 | FB-1.50,286.36 \tiny{(38.31)},43.68 \tiny{(0.53)},330.04 \tiny{(38.63)},48.0
12,NZ-64 | FB-0.75,293.67 \tiny{(37.93)},40.61 \tiny{(0.57)},334.29 \tiny{(38.25)},48.0


In [19]:
from functools import reduce

order = [
    "2021-02-03-PTB-latent32-FB-0.00-run-14:32:09",
    "2021-02-02-PTB-latent32-FB-0.25-run-13:16:32",
    "2021-02-03-PTB-latent32-FB-0.5-run-09:31:02",
    "2021-02-02-PTB-latent32-FB-0.75-run-12:44:32",
    "2021-02-03-PTB-latent32-FB-1.0-run-11:43:17",
    "2021-02-03-PTB-latent32-FB-1.50-run-12:13:36",
    "2021-02-03-PTB-latent32-autoencoder-run-17:30:41",
    "2021-02-03-PTB-latent64-FB-0.00-run-17:14:10",
    "2021-02-02-PTB-latent64-FB-0.25-run-13:17:02",
    "2021-02-03-PTB-latent64-FB-0.50-run-12:29:58",
    "2021-02-02-PTB-latent64-FB-0.75-run-13:16:36",
    "2021-02-03-PTB-latent64-FB-1.0-run-13:06:00",
    "2021-02-03-PTB-latent64-FB-1.50-run-13:22:14",
    "2021-02-03-PTB-latent64-autoencoder-run-18:25:57"
]

nz_32_c = {
    "FB-0.00":"#AED6DE",
    "FB-0.25":"#86C1CD",
    "FB-0.50":"#5DADBD", 
    "FB-0.75":"#3598AC", 
    "FB-1.00":"#287281", 
    "FB-1.50":"#1B4C56",
    "autoencoder":"#0D262B"
}

nz_64_c = {
    "FB-0.00":"#FBC5B9",
    "FB-0.25":"#F8A895",
    "FB-0.50":"#F68B72", 
    "FB-0.75":"#F46E4F", 
    "FB-1.00":"#B7533B", 
    "FB-1.50":"#7A3728",
    "autoencoder":"#3D1C14", 
}



dfs = [df_rate, df_au, df_acc, df_ppl] #
df_final = reduce(lambda left,right: pd.merge(left, right, on='index', how="outer"), dfs)
df_final = df_final.rename(columns = {'index':'Run'})
df_final = df_final.set_index("Run")

# ind = []
# for o in order:
#     for name in list(df_final.index.values):
#         if name == get_clean_name(o):
#             i = list(df_final.index.values).index(get_clean_name(o))
#             ind.append(get_clean_name(o))
# df_final.sort_values(by=[0, 1, 2, 3, 4, 5,6 ,7 ,8, 9])
df_final

,Distortion,Rate,-ELBO,Target rate (FB x NZ),Active Units,Accuracy,Perplexity
Run,,,,,,,
NZ-32 | FB-0.25,286.38 \tiny{(26.06)},6.01 \tiny{(0.08)},292.38 \tiny{(26.09)},8.0,32 / 32,0.05 \tiny{(0.07)},6.69e+03 \tiny{(9.33e+02)}
NZ-64 | FB-0.25,284.03 \tiny{(27.61)},11.09 \tiny{(0.14)},295.12 \tiny{(27.71)},16.0,64 / 64,0.06 \tiny{(0.07)},6.18e+03 \tiny{(9.63e+02)}
NZ-32 | FB-0.50,295.39 \tiny{(30.12)},13.10 \tiny{(0.17)},308.49 \tiny{(30.22)},16.0,32 / 32,0.06 \tiny{(0.08)},8.03e+03 \tiny{(1.40e+03)}
NZ-32 | FB-0.00,308.59 \tiny{(29.11)},0.00 \tiny{(0.00)},308.59 \tiny{(29.11)},0.0,0 / 32,0.04 \tiny{(0.07)},1.78e+05 \tiny{(2.41e+04)}
NZ-64 | FB-0.00,309.21 \tiny{(29.17)},0.00 \tiny{(0.00)},309.22 \tiny{(29.17)},0.0,0 / 64,0.04 \tiny{(0.07)},1.82e+05 \tiny{(2.48e+04)}
NZ-32 | FB-0.75,298.88 \tiny{(31.44)},20.52 \tiny{(0.26)},319.40 \tiny{(31.57)},24.0,32 / 32,0.07 \tiny{(0.10)},1.08e+04 \tiny{(1.98e+03)}
NZ-64 | FB-0.50,296.16 \tiny{(29.72)},24.45 \tiny{(0.34)},320.61 \tiny{(29.92)},32.0,64 / 64,0.07 \tiny{(0.11)},9.76e+03 \tiny{(1.83e+03)}
NZ-32 | FB-1.00,297.86 \tiny{(33.61)},29.51 \tiny{(0.36)},327.37 \tiny{(33.80)},32.0,32 / 32,0.11 \tiny{(0.16)},1.15e+04 \tiny{(2.45e+03)}
NZ-32 | FB-1.50,286.36 \tiny{(38.31)},43.68 \tiny{(0.53)},330.04 \tiny{(38.63)},48.0,32 / 32,0.14 \tiny{(0.20)},1.47e+04 \tiny{(3.38e+03)}


In [20]:
print(df_final.to_latex())

\begin{tabular}{llllrlll}
\toprule
{} &             Distortion &                  Rate &                  -ELBO &  Target rate (FB x NZ) & Active Units &            Accuracy &                  Perplexity \\
Run                 &                        &                       &                        &                        &              &                     &                             \\
\midrule
NZ-32 | FB-0.25     &  286.38 \textbackslash tiny\{(26.06)\} &    6.01 \textbackslash tiny\{(0.08)\} &  292.38 \textbackslash tiny\{(26.09)\} &                    8.0 &      32 / 32 &  0.05 \textbackslash tiny\{(0.07)\} &  6.69e+03 \textbackslash tiny\{(9.33e+02)\} \\
NZ-64 | FB-0.25     &  284.03 \textbackslash tiny\{(27.61)\} &   11.09 \textbackslash tiny\{(0.14)\} &  295.12 \textbackslash tiny\{(27.71)\} &                   16.0 &      64 / 64 &  0.06 \textbackslash tiny\{(0.07)\} &  6.18e+03 \textbackslash tiny\{(9.63e+02)\} \\
NZ-32 | FB-0.50     &  295.39 \textbackslash tiny\{(30.12